In [1]:
import tensorflow as tf
from forward_kinematics import forward_kinematics
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

In [2]:
def cos(t):
    return np.cos(np.deg2rad(t))

# Create data

In [3]:
# Generate 220000 samples
x_samples=[]
y_samples=[]
l1=104.
#for i in range(220000):
for i in range(220000):
    T1=np.random.randint(0,91)
    T2=np.random.randint(0,91)
    T3=np.random.randint(0,91)
    T4=np.random.randint(0,91)        
    fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
    dx=fk[0]
    dy=fk[1]
    dz=fk[2]
    r = np.sqrt(dx**2+dy**2)

    x_samples.append([r,dz])
    y_samples.append([T2])
    
x_samples=np.array(x_samples)
y_samples=np.array(y_samples,dtype=np.float32)

In [4]:
np.max(x_samples[:,0])

368.3164400349243

In [5]:
#dx,dy,dz=x_samples[:,0],x_samples[:,1],x_samples[:,2]
dr,dz=x_samples[:,0],x_samples[:,1]


## Data normalization

In [6]:
# Step 1: Determine the Range
#min_dx, max_dx = min(dx), max(dx)
min_r, max_r = min(dr), max(dr)
#min_dy, max_dy = min(dy), max(dy)
min_dz, max_dz = min(dz), max(dz)

# Step 2: Normalize Each Variable
#normalized_dx = [(x - min_dx) / (max_dx - min_dx) for x in dx]
#normalized_dy = [(y - min_dy) / (max_dy - min_dy) for y in dy]
normalized_dz = [(z - min_dz) / (max_dz - min_dz) for z in dz]
normalized_r = [(r - min_r) / (max_r - min_r) for r in dr]
# Normilize Y samples
#normalized_y =np.array([t2/90 for t2 in y_samples])
normalized_y =np.array([t2 for t2 in y_samples])

In [7]:
#normalized_dx[0],normalized_dy[0],normalized_dz[0]
normalized_r[0],normalized_dz[0]

(0.7085465607623419, 0.20253164556962025)

In [8]:
#normalized_x=np.transpose(np.array([normalized_dx,normalized_dy,normalized_dz]))
normalized_x=np.transpose(np.array([normalized_r,normalized_dz]))
normalized_x

array([[0.70854656, 0.20253165],
       [0.83569514, 0.41772152],
       [0.55557914, 0.78797468],
       ...,
       [0.61137023, 0.43987342],
       [0.55912371, 0.07594937],
       [0.84221915, 0.34810127]])

In [9]:
#normalized_y=np.transpose(np.array([normalized_dT2,normalized_dT3,normalized_dT4]))
#normalized_y

# Split data

In [10]:
X_train = normalized_x[:200000]
X_val = normalized_x[200000:]

In [11]:
y_train = normalized_y[:200000]
y_val = normalized_y[200000:]

In [12]:
X_train.shape

(200000, 2)

# Create model

In [17]:
inputs = layers.Input(shape=(2,))
x=layers.Dense(32, activation='tanh')(inputs)
x=layers.Dense(64,activation='tanh')(x)
x=layers.Dense(32, activation='tanh')(x)
out=layers.Dense(91,activation="softmax")(x)

model_1=tf.keras.Model(inputs,out)

In [18]:
# Compile model
loss = tf.keras.losses.sparse_categorical_crossentropy
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100, restore_best_weights=True)
model_1.compile(loss=loss, optimizer=optimizer, metrics=tf.metrics.RootMeanSquaredError())

In [19]:
np.array(y_train)

array([[75.],
       [46.],
       [ 0.],
       ...,
       [43.],
       [19.],
       [36.]], dtype=float32)

In [20]:
# Train model
model_1.fit(X_train, y_train, 
            epochs=10,
            validation_data=(X_val,y_val), 
            callbacks=[early_stop])

Epoch 1/10
6250/6250 [==============================] - 5s 664us/step - loss: 3.6825 - root_mean_squared_error: 52.0928 - val_loss: 3.4792 - val_root_mean_squared_error: 52.1438
Epoch 2/10
6250/6250 [==============================] - 4s 651us/step - loss: 3.4431 - root_mean_squared_error: 52.0927 - val_loss: 3.4220 - val_root_mean_squared_error: 52.1438
Epoch 3/10
6250/6250 [==============================] - 4s 654us/step - loss: 3.3970 - root_mean_squared_error: 52.0928 - val_loss: 3.3713 - val_root_mean_squared_error: 52.1438
Epoch 4/10
6250/6250 [==============================] - 4s 643us/step - loss: 3.3557 - root_mean_squared_error: 52.0927 - val_loss: 3.3379 - val_root_mean_squared_error: 52.1438
Epoch 5/10
6250/6250 [==============================] - 4s 646us/step - loss: 3.3188 - root_mean_squared_error: 52.0927 - val_loss: 3.2982 - val_root_mean_squared_error: 52.1438
Epoch 6/10
6250/6250 [==============================] - 4s 657us/step - loss: 3.2888 - root_mean_squared_error

In [21]:
np.array([0.123]).shape

(1,)

# Predict using model

In [22]:
T1=np.random.randint(0,91)
T2=np.random.randint(0,91)
T3=np.random.randint(0,91)
T4=np.random.randint(0,91)    
print(T1,T2,T3,T4)
fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
fk

63 83 9 11


array([ 165., -323.,   65.])

In [23]:
#predict_dx=(fk[0] - min_dx) / (max_dx - min_dx) 
#predict_dx

In [24]:
predict_r=(np.sqrt(fk[0]**2+fk[1]**2) - min_r) / (max_r - min_r) 
predict_r

0.98476119017721

In [25]:
#predict_dy=(fk[1] - min_dy) / (max_dy - min_dy) 
#predict_dy

In [26]:
predict_dz=(fk[2] - min_dz) / (max_dz - min_dz) 
predict_dz

0.370253164556962

In [37]:
output=model_1.predict([[predict_r,predict_dz]])
np.argmax(output)

1/1 [==============================] - 0s 16ms/step


86

In [28]:
T1= np.arctan(fk[1]/fk[0])
T1=np.degrees(-T1)
T1=np.round(T1)
T1

63.0

In [29]:
forward_kinematics(T1,output[0],output[1],output[2])[:3,3]

array([  0.,  -0., 463.])

In [30]:
fk

array([ 165., -323.,   65.])

In [31]:
out=np.round(output*90)[0]
out

0.0

In [32]:
def cos(t):
    return np.cos(np.deg2rad(t))
def sin(t):
    return np.sin(np.deg2rad(t))
def finder(fk, i):
    dx,dy,dz=fk
    eT2=i
    l1=104.
    l2=89.
    l3=175.
    r = np.sqrt(dx**2+dy**2)
    r1 = cos(eT2)*l1
    r2 = r-r1
    t = sin(eT2)*l1
    t1=t-(dz-95)
    if(t1!=0):        
        alpha1=np.arctan(r2/t1)
        alpha1=np.degrees(alpha1)
    else:
        alpha1=0
    b=np.sqrt(t1**2+r2**2)
    operation1=(l2**2+b**2-l3**2)/(2*l2*b)
    operation2=(l2**2+l3**2-b**2)/(2*l2*l3)
    if(int(operation1*100)>100 or int(operation1*100)<-100):
        raise Exception() 
    if(int(operation1*100)==100 or int(operation1*100)==-100):
        operation1=np.round(operation1)
        
    if(int(operation2*100)>100 or int(operation2*100)<-100):
        raise Exception() 
    if(int(operation2*100)==100 or int(operation2*100)==-100):
        operation2=np.round(operation2)

    alpha2=np.arccos(operation1)
    alpha2=np.degrees(alpha2)
    if(t1>0):
        T3=180-(alpha1+alpha2+i)
        T3=np.round(T3)
    else:
        T3=alpha1+alpha2+i
        T3=abs(np.round(T3))
    T4=np.arccos(operation2)

    T4=np.degrees(T4)
    T4=180-T4
    T4=np.round(T4)
    T1= np.arctan(dy/dx)
    T1=np.degrees(-T1)
    T1=np.round(T1)
    #if T3>90 or T4>90:
     #   raise Exception()
    return [T1,T2,T3[0], T4[0]]

In [33]:
predicted=finder(fk,out)

IndexError: invalid index to scalar variable.

In [ ]:
predict_fk=forward_kinematics(*predicted)[:3,3]
predict_fk

In [ ]:
fk